<a href="https://colab.research.google.com/github/RiteshDataLab/ML_intern/blob/main/ml1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

!pip install pandas
!pip install scikit-learn
#libraries
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import numpy as np

In [17]:
df = pd.read_csv('10000.csv')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   image_index                    10000 non-null  int64 
 1   type                           10000 non-null  object
 2   Title                          10000 non-null  object
 3   X-tick                         10000 non-null  object
 4   Y-tick                         10000 non-null  object
 5   legend                         10000 non-null  object
 6   X-label                        10000 non-null  object
 7   Ylabel                         10000 non-null  object
 8   image_index.1                  10000 non-null  int64 
 9   question_id                    10000 non-null  int64 
 10  question_string                10000 non-null  object
 11  ChartFacts                     9942 non-null   object
 12  Labels                         10000 non-null  object
 13  an

In [19]:
df.shape

(10000, 16)

In [20]:
df.head()

,image_index,type,Title,X-tick,Y-tick,legend,X-label,Ylabel,image_index.1,question_id,question_string,ChartFacts,Labels,answer,template,copy_of_true_all_true_answers
0,0,dot_line,Percent of total population of Singapore who s...,"{'1993', '1997', '1996', '1995', '1994'}","{'20', '40', '0', '80', '60'}","['Age 65(female)', 'Age 65(male)']",Year,Population(% of cohort),0,4995932,What is the Population(% of cohort) of Age 65(...,81.03166 is the Population(% of cohort) of Age...,Yes,81.03166,data_retrieval,81.03166
1,0,dot_line,Percent of total population of Singapore who s...,"{'1993', '1997', '1996', '1995', '1994'}","{'20', '40', '0', '80', '60'}","['Age 65(female)', 'Age 65(male)']",Year,Population(% of cohort),0,4995945,What is the difference between the Population(...,-0.6583 is the difference between the Populati...,Yes,-0.6583,arithmetic,-0.6583
2,0,dot_line,Percent of total population of Singapore who s...,"{'1993', '1997', '1996', '1995', '1994'}","{'20', '40', '0', '80', '60'}","['Age 65(female)', 'Age 65(male)']",Year,Population(% of cohort),0,4995987,What is the ratio of the Population(% of cohor...,0.9889 is the ratio of the Population(% of coh...,Yes,0.9889,comparison,0.9889
3,0,dot_line,Percent of total population of Singapore who s...,"{'1993', '1997', '1996', '1995', '1994'}","{'20', '40', '0', '80', '60'}","['Age 65(female)', 'Age 65(male)']",Year,Population(% of cohort),0,4995986,What is the ratio of the Population(% of cohor...,0.9879 is the ratio of the Population(% of coh...,Yes,0.9879,comparison,0.9879
4,0,dot_line,Percent of total population of Singapore who s...,"{'1993', '1997', '1996', '1995', '1994'}","{'20', '40', '0', '80', '60'}","['Age 65(female)', 'Age 65(male)']",Year,Population(% of cohort),0,4995948,What is the difference between the Population(...,-1.4783 is the difference between the Populati...,Yes,-1.4783,arithmetic,-1.4783


In [21]:
# Prepare the data
df['labels'] = df['ChartFacts'].apply(lambda x: 1 if x == 'Yes' else 0)
train_texts, val_texts, train_labels, val_labels = train_test_split(df['question_string'].tolist(), df['labels'].tolist(), test_size=0.2)

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [22]:
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512)

In [23]:
train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

In [24]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [27]:
train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)


In [11]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
!pip install pytorch-accelerated -U
!pip install transformers[torch]
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [8]:
data_collator = DataCollatorWithPadding(tokenizer)

In [9]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [28]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

Step,Training Loss
10,0.893700
20,0.655900
30,0.404200
40,0.305500
50,0.238000
60,0.191000
70,0.115000
80,0.061800
90,0.027800
100,0.014400


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 3.423684347581002e-06,
 'eval_accuracy': 1.0,
 'eval_f1': 0.0,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 242.9407,
 'eval_samples_per_second': 8.232,
 'eval_steps_per_second': 1.029,
 'epoch': 3.0}

In [29]:
# Function to predict "Yes" or "No"
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return 'Yes' if probs.argmax() == 1 else 'No'

In [32]:
# Example usage
example_text = "What is the ratio of the Population(% of cohort) of Age 65(female) in 1993 to that in 1994?"
prediction = predict(example_text)
print(f"Prediction: {prediction}")

Prediction: No


In [35]:
def compute_metrics(preds, labels):
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }